# Importing required modules and programs

In [ ]:
from dbconnector import connection
from client import Client
import time
import threading
import hashlib
import os

# Connecting To Database

In [ ]:
mycursor,db = connection("192.168.1.11")
print("[SUCCESS] Connected to database succesfully...")

# Making query in Database

In [ ]:
Filename = input("Enter Filename: ")

print("[SEARCHING] Searching for file in database...")

mycursor.execute(f"""SELECT u.PublicIP, f.SIZE, f.CHECKSUM FROM filedatabase f INNER JOIN userdatabase u USING (USERNAME) 
WHERE f.FILENAME = '{Filename}' """)

listof_IPs = []

size = 0
checksum = ""

for x in mycursor:
    listof_IPs.append(x[0])
    size = int(x[1])
    checksum = x[2]
print("[COMPLETE] Search Complete...")

# Connecting to peers

In [ ]:
number_of_peers = len(listof_IPs)
inc = size//number_of_peers
offset = 0
offset_lookup = {}
for i,IP in enumerate(listof_IPs):
    if IP not in offset_lookup:
        offset_lookup[IP] = dict()
    offset_lookup[IP]['start'] = offset
    if i == number_of_peers-1:
        offset_lookup[IP]['end'] = size
    else:
        offset_lookup[IP]['end'] = offset + inc
        offset += inc
client_lookup = {}
for IP in listof_IPs:
    client_lookup[IP] = Client(IP,offset_lookup[IP]['start'],offset_lookup[IP]['end'],Filename)
    
for IP in client_lookup:
    client_lookup[IP].handshake()



# Receiving Chunks

In [ ]:
location = input("Enter Location where you want to download file: ")

tick = time.time()

print("[DOWNLOADING] Started downloading...")
threads=[]
for IP in client_lookup:
    t=threading.Thread(target=client_lookup[IP].receiving, args=[location])
    t.start()
    threads.append(t)
for thread in threads:
    thread.join()

clock= time.time()
print(f"[COMPLETED] Finished downloading in {(clock-tick):.2f} seconds!")

# Joining Chunks

In [ ]:
print("[JOINING] Joining chunks...")
target = Filename
count = 0
for IP in client_lookup:
    with open(IP,'rb') as tempfile:
        data  = tempfile.read()
        with open(target,'ab') as dest:
            dest.write(data)
    count += 1
    print(f"{count} of {number_of_peers} chunks joined...")
    os.remove(IP)
print("[COMPLETE] Joining complete...")

# Performing Checksum

In [ ]:
print("[VERIFYING] Verfiying download...")

hasher = hashlib.md5()

with open(target,'rb') as Source:
    content = Source.read()
    hasher.update(content)
recvd_check=hasher.hexdigest()

if recvd_check != checksum:
     print("[FAILED] Please try again...")
else:
     print("[SUCCESSFUL] Downloaded Succesfully...")